# This Notebook is Training word2vec Model with content_id
First of all, Thank you Kaggle, the Competition Host and competitor!!  
This competition gives me a lot of learning.  
  
And I'm impressed with Kaggle's culture of sharing ideas with everyone.

In this competition, I learned how to use the new idea of word2vec from [this discussion](https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/209576) and other.  
Thank you @ML_Bear for sharing ideas!!

So I decided to try implementing this idea.  


This is my first implementation of word2vec, so I may make some mistakes.  
But I'll share it in this notebook!!  

If you have any concerns about memory savings or processing speed, please let me know.  

# Import

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1

In [ ]:
import numpy as np
import random
import pandas as pd
import datatable as dt
import gc
from tqdm.notebook import tqdm
from collections import defaultdict

# Read Data

In [ ]:
data_types_dict = {
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id':'int8',
    'answered_correctly': 'int8'
}

train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()

In [ ]:
# delete lecture rows
train_df = train_df[~train_df['content_type_id']]
train_df.drop('content_type_id', axis=1, inplace=True)
gc.collect()

# Data to Sentence

In [ ]:
# Added the 6th digit to question_id to distinguish answered_correctly
train_df['word'] = train_df['content_id'] + train_df['answered_correctly'] * 100000

In [ ]:
# memory error
# sentences = {}

# for _, group in enumerate(tqdm(train_df.groupby('user_id'))):
#     sentences[group[0]] = list(group[1]['word'].apply(str))

In [ ]:
sentences = defaultdict(list)

for _,row in enumerate(tqdm(train_df[['user_id', 'word']].values)):
    sentences[row[0]].append(str(row[1]))

In [ ]:
del train_df
sentences = list(sentences.values())
gc.collect()

# word2vec

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

model = Word2Vec(sentences,  sg=1, size=100, window=5, min_count=1, sample=0)
model.wv.save_word2vec_format("vec.pt", binary=True)

del model, sentences
gc.collect()

In [ ]:
wv = KeyedVectors.load_word2vec_format('vec.pt', binary=True)
print('most_similar')
print(wv.most_similar('105692'))

print('get_vector')
print(wv.get_vector('105692'))

[This Notebook](https://www.kaggle.com/imazekishota/riiid-lgbm-with-word2vec) is LGBM training code.